


You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```

{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [1]:
#Necessary packages
import pandas as pd


In [2]:
#Read all the files
employee_addresses = pd.read_excel("employee_information.xlsx", sheet_name = 0)

employee_roles = pd.read_json("employee_roles.json").T
employee_roles.reset_index(drop=False, inplace=True)

office_address = pd.read_csv("office_addresses.csv")


#All these is me trying to insert a header row that will label the unlabeled columns
# Specify the new column names
new_column_names = [
    'employee_id',
    'employee_last_name',
    'employee_first_name',
    'emergency_contact',
    'emergency_contact_number',
    'relationship'
]

# Read the Excel file without header
emergency_contacts = pd.read_excel("employee_information.xlsx", sheet_name=1, header=None, names = new_column_names)

In [3]:
# Create an empty DataFrame
employees_final = pd.DataFrame()

In [4]:
#Here I am adding the first 7 columns into the new data frame
employees_final = pd.concat([employees_final, employee_addresses], ignore_index = True)

#Swapping the first name with the last name
employees_final = employees_final.iloc[:, [0,2,1] + list(range(3, len(employees_final.columns)))]

In [5]:
#Adding the next 3 columns to the final file.
next_3 = emergency_contacts.iloc[:, 3:6]

employees_final = pd.concat([employees_final, next_3], axis = 1)

In [6]:
#Now adding the next 3 columns from the json file
cols_11_13 = employee_roles[['monthly_salary', 'team', 'title']]

employees_final = pd.concat([employees_final, cols_11_13], axis = 1)

In [7]:
# Merge employees with offices based on employee_country and office_country
employees_final = pd.merge(employees_final, office_address, how='left', left_on='employee_country', right_on='office_country')

In [8]:
employees_final.fillna("Remote", inplace = True)

C:\Users\HP\AppData\Local\Temp\ipykernel_15312\3446274966.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Remote' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  employees_final.fillna("Remote", inplace = True)


In [9]:
#Setting the employee_id as the index
employees_final = employees_final.set_index(employees_final.columns[0])

In [10]:
employees_final.head()

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote


``CHAPTER 2: ADVANCED ETL TECHNIQUES.``

**A). Advanced data transformation with pandas**

Filling missing values with pandas:

When building data pipelines, it's inevitable that you'll stumble upon missing data. In some cases, you may want to remove these records from the dataset. But in others, you'll need to impute values for the missing information. In this exercise, you'll practice using pandas to impute missing test scores.

Grouping data with pandas

The output of a data pipeline is typically a "modeled" dataset. This dataset provides data consumers easy access to information, without having to perform much manipulation. Grouping data with pandas helps to build modeled datasets,

Applying advanced transformations to DataFrames

pandas has a plethora of built-in transformation tools, but sometimes, more advanced logic needs to be used in a transformation. The apply function lets you apply a user-defined function to a row or column of a DataFrame, opening the door for advanced transformation and feature generation.

![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [11]:
brands = pd.read_csv("brands.csv")
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

In order to complete the project you will need to merge the datasets and drop null values, along with answering the questions below.

In [12]:
#Merge the data sets
df = pd.merge(brands, finance, on='product_id', how='outer')
df = pd.merge(df, info, on='product_id', how='outer')
df = pd.merge(df, reviews, on='product_id', how='outer')

# Set 'product_id' as the first column
df = df[['product_id'] + [col for col in df.columns if col != 'product_id']]

#Drop the null value
df = df.dropna()

df.head()

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
0,130690-017,Nike,0.00,159.95,0.0,6909.84,Air Jordan 12 Retro,An all-time favourite among players and sneake...,4.5,24.0
1,133000-106,Nike,0.00,119.95,0.0,0.00,Air Jordan OG,"First released in '98, the Air Jordan OG was d...",0.0,0.0
2,280648,Adidas,29.99,29.99,0.0,2915.03,Men's Originals Summer Adilette Slippers,"From 72' until now, these adidas Originals San...",4.2,54.0
3,288022,Adidas,29.99,29.99,0.0,5128.29,Men's Originals Summer Adilette Slides,The adidas Originals Adilette slip ons for men...,3.3,95.0
4,310805-137,Nike,0.00,159.95,0.0,64203.93,Air Jordan 10 Retro,"Featuring soft, lightweight cushioning, the Ai...",4.7,223.0


1. What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".

In [13]:
# Calculate listing price quartiles for all products
quartiles = pd.qcut(df['listing_price'], q=4, labels=['Budget', 'Average', 'Expensive', 'Elite'])

# Add quartiles as a new column to the DataFrame
df['listing_price_label'] = quartiles

# Filter the DataFrame to separate Adidas and Nike products
adidas = df[df['brand'] == 'Adidas']
nike = df[df['brand'] == 'Nike']

# Calculate volume and average revenue for Adidas products based on quartiles
adidas_volume_revenue = adidas.groupby('listing_price_label').agg(volume=('listing_price_label', 'count'), mean_revenue=('revenue', 'mean'))

# Calculate volume and average revenue for Nike products based on quartiles
nike_volume_revenue = nike.groupby('listing_price_label').agg(volume=('listing_price_label', 'count'), mean_revenue=('revenue', 'mean'))

C:\Users\HP\AppData\Local\Temp\ipykernel_15312\1519729351.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adidas_volume_revenue = adidas.groupby('listing_price_label').agg(volume=('listing_price_label', 'count'), mean_revenue=('revenue', 'mean'))
C:\Users\HP\AppData\Local\Temp\ipykernel_15312\1519729351.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  nike_volume_revenue = nike.groupby('listing_price_label').agg(volume=('listing_price_label', 'count'), mean_revenue=('revenue', 'mean'))


Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

In [14]:
# adidas_vs_nike = pd.DataFrame(columns = ["brand", "price_label", "num_products","mean_revenue"])

# Concatenate Adidas and Nike results
adidas_vs_nike = pd.concat([adidas_volume_revenue, nike_volume_revenue], keys=['Adidas', 'Nike']).reset_index()
adidas_vs_nike.rename(columns={'level_0': 'brand'}, inplace=True)
adidas_vs_nike['mean_revenue'] = adidas_vs_nike['mean_revenue'].round(2)

adidas_vs_nike.rename(columns={'volume': 'num_products', 'listing_price_label': 'price_label'}, inplace=True)


adidas_vs_nike.head()

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33


2. Do any differences exist between the word count of a product's description and its mean rating?

Split product description length into bins of 100 characters and calculate the average rating and number of reviews.

In [15]:
# Calculate the length of each product's description
df['description_length'] = df['description'].str.len()

# Split description lengths into bins of 100 characters
# bins = range(0, df['description_length'].max() + 101, 100)
df['description_length'] = pd.cut(df['description_length'], bins = [0, 100, 200, 300, 400, 500, 600, 700], labels = ["100", "200", "300", "400", "500", "600", "700"])

# Calculate average rating and number of reviews for each bin
description_lengths = df.groupby('description_length').agg(
    mean_rating=('rating', 'mean'),
    num_reviews=('reviews', 'count')
)

# Round numeric values to two decimal places
description_lengths = description_lengths.round(2)

C:\Users\HP\AppData\Local\Temp\ipykernel_15312\2342081473.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  description_lengths = df.groupby('description_length').agg(


Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.

In [16]:
# Rename the index column
description_lengths.index.name = 'description_length'

# Reset index to convert the index into a column
description_lengths.reset_index(inplace=True)

# Create the DataFrame with the specified columns
# description_lengths = description_lengths[['description_length', 'mean_rating', 'num_reviews']]

In [17]:
description_lengths

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15
